In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [4]:
train_data, train_label = load_wine(return_X_y=True)

#### Keeping only the first two classes to create a binary problem as instructed
train_data = np.array(train_data[:(59+71)])
train_label = np.array(train_label[:(59+71)])

train_data = (train_data - train_data.mean(axis=0))/(train_data.max(axis=0)-train_data.min(axis=0))

In [5]:
logist_regr = LogisticRegression(solver='sag', penalty=None, max_iter=20000).fit(train_data, train_label)
ideal_loss = log_loss(train_label, logist_regr.predict_proba(train_data))

print(ideal_loss)

0.0002656488462351898


In [6]:
def steepest_coord_descent(tr_data, tr_label, weights, max_iter=100000, lr=0.01):

    loss_steepest_cd = []

    for iter in range(max_iter):
        #pred = 1./(1. + np.exp(-1. * np.matmul(weights.T, train_data.T)))
        pred = 1./(1. + np.exp(-1. * np.dot(weights.T, tr_data.T)))

        #print(pred)
        iter_loss = -(1./len(tr_data)) * np.sum(
            tr_label * np.log(pred) +
            (1 - tr_label) * np.log(1. - pred)
        )
        loss_steepest_cd.append(iter_loss)
        #print(iter_loss)

        #partial_grads = np.sum(np.matmul((pred - train_label), train_data), axis=0) / len(train_data)
        partial_grads = list(np.dot((pred - tr_label), tr_data)[0])
        #print(partial_grads)
        steepest_coord = np.argmax(np.abs(partial_grads))
        weights[steepest_coord] = weights[steepest_coord] - lr * partial_grads[steepest_coord]
        #print(weights[steepest_coord])

    print(loss_steepest_cd[-1])
    return weights, loss_steepest_cd

In [7]:
weights = np.zeros([len(train_data[0]), 1])
lr = 0.02
max_iter = 800000
loss_steepest_cd = []

weights_steepest_cd, loss_steepest_cd = steepest_coord_descent(train_data, train_label, weights, max_iter, lr)

0.0003024634101822091
